In [ ]:
import os
import re
from datetime import datetime
from ltlcross_runner import LtlcrossRunner
from tools_hier import get_tools, ltl3dra_tools, full_tools, tool_order
from evaluation_utils import sort_by_tools, split_cols
import pandas as pd
import spot
spot.setup()

# Comparison of LTL to deterministic automata translators 

$\newcommand{\F}{\mathsf{F}}\newcommand{\G}{\mathsf{G}}\newcommand{\U}{\mathsf{U}}\newcommand{\X}{\mathsf{X}}$
## Tools
We have 
* tools that translate \[fragments\] of LTL into (generalized) Rabin automata:
 - [Rabinizer 3.1](https://www7.in.tum.de/~kretinsk/rabinizer3.html)
 - [Rabinizer 4](https://www7.in.tum.de/~kretinsk/rabinizer4.html)
 - [LTL3DRA](https://sourceforge.net/projects/ltl3dra/) v.0.2.6 \[LTL$\smallsetminus\G$($\U\X$)\]
* tools that chain translation of LTL into cut-deterministic Büchi automata (aka LDBA) or into deterministic Rabin automata with a construction that create deterministic parity automaton. They avoid Safra-like determinization. Both are from the [owl/Rabinizer4 library](https://www7.in.tum.de/~kretinsk/rabinizer4.html)
 - ltl2dpa --mode=ldba
 - ltl2dpa --mode=rabinizer (relies on transition-based index appearance record)
* tools that perform Safra-based determinization of N\[G\]BA into {Rabin/Parity} automata that use intermediete NBA
 - [ltl2dstar](http://ltl2dstar.de) \[NBA only\] {Rabin)}
 - autfilt from [Spot](https://spot.lrde.epita.fr/) 2.5 {parity}
 - here we use the following LTL2TGBA/LTL2NBA translators:
   - ltl2tgba from [Spot](https://spot.lrde.epita.fr/) 2.5
   - [LTL3BA](https://sourceforge.net/p/ltl3ba/) v. 1.1.3; we use LTL3BA in two settings:
     - `ltl3ba` {LTL3BA}
     - `ltl3ba -d` {LTL3BAd} which prefer more deterministic automata to smaller ones
 - we further use experimentaly for autfilt the LTL to Emerson-Lei automata translator
   - [LTL3TELA](https://github.com/jurajmajor/ltl3tela) v.1.1.1
* Safra's based determinization into deterministic parity automata that uses information about the LTL formula
 - `ltl2tgba -DG` from [Spot](https://spot.lrde.epita.fr/) 2.5

In [ ]:
runners = {}
cols=["states","transitions","acc","time","nondet_states"]
for source in ('literature','random'):
    for t in ('full','ltl3dra'):
        name = '{}_{}'.format(source,t)
        tools = full_tools if t=='full' else ltl3dra_tools
        runners[name] = \
            LtlcrossRunner(tools,\
                    res_filename='data/{}.csv'.format(name),\
                    formula_files=['formulae/{}.ltl'.format(name)],\
                    cols=cols)

name = 'random_fg'
tools = ltl3dra_tools
runners[name] = LtlcrossRunner(tools,\
                    res_filename='data/{}.csv'.format(name),\
                    formula_files=['formulae/{}.ltl'.format(name)],\
                    cols=cols)

In [ ]:
for name, r in runners.items():
    print('{}: Working on {}'.format(datetime.now().strftime('[%d.%m.%Y %T]'),name))
    r.name = name
    r.parse_results()
    r.na_incorrect()
    r.orig_count = len(r.values)
    r.clean_count = len(r.values.dropna())
    r.compute_sbacc()

In [ ]:
for name, r in runners.items():
    print('The benchmark {} started with {} formulae and has valid data for {} formulae'.format(name,r.orig_count,r.clean_count))
    r.errors = {}
    for e in ['timeout', 'parse error', 'incorrect', 'crash', 'no output']:
        s = r.get_error_count(e).sum()
        if not pd.isna(s):
            r.errors[e] = r.get_error_count(e).sum()
    s = pd.Series(r.errors, name='count')
    s.index.name = 'error type'
    display(s.reset_index())

## Gather data

In [ ]:
def compute_runner(self,cols=['states','acc','time'],
                   tool_subset=None, split_col_names=True, col_lev_names=None,
                   counts = True):
    '''Returns a dataframe with rows indexed by tools in
    `tool_subset (or `self.tools.keys()`) and columns by
    cols. The values are cummulative sums for respective
    tool and column.'''
    if tool_subset is None:
        tool_subset = self.tools.keys()
    name = self.name
    name = name.replace('ltl3dra','ltl-gux')
    if counts:
        name = '{} ({})'.format(name,self.clean_count)
    col_names = ['{}_{}'.format(name,c) for c in cols]
    df = pd.DataFrame(columns=col_names,index=tool_subset)
    for col in cols:
        df['{}_{}'.format(name,col)] = self.cummulative(col).map(lambda x: '%0.0f' % x)
    if split_col_names:
        res = split_cols(df,axis=1,symbol='_',names=col_lev_names)
    else:
        res = df
    return res 

In [ ]:
display(compute_runner(r).head())
display(compute_runner(r,split_col_names=False).head())
compute_runner(r,col_lev_names=['source','fragment','metric']).head()

In [ ]:
data = []
order = [
    ('literature','full'),
    ('literature','ltl3dra'),
    ('random','full'),
    ('random','ltl3dra'),
    ('random','fg'),
]
for o in order:
    data.append(compute_runner(runners['{}_{}'.format(o[0],o[1])]))
res = pd.concat(data,axis=1)

In [ ]:
def fix_acc(acc,sb=False):
    acc = acc.replace('TGBA','TGB').replace('NBA','SB')
    acc = acc.replace('DPA','DTPA').replace('DTPA','TP')
    acc = acc.replace('DSRA','SR').replace('DTGRA','TGR')
    acc = acc.replace('DTELA','TEL').replace('TEL.TP','TEL.TEL')
    if sb:
        acc = re.sub(r'T([^T]+$)', r'S\1', acc)
    return acc

In [ ]:
def fix_tools(tool):
    tool = tool.replace('ltl2dstar(NBA)',
                        '\parbox[c]{1.3cm}{\centering ltl2dstar NBA}')
    tool = tool.replace('Spot',
                        #'\rotatebox[origin=c]{90}{Spot (autfilt)}')
                        '\parbox[c]{1.3cm}{\centering Spot autfilt}')
    tool = tool.replace('R3','Rabinizer 3').replace('R4','Rabinizer 4')
    return tool

In [ ]:
def fix_fragment(fr,vertical=False):
    fr = fr.replace('full','full LTL')
    fr = fr.replace('ltl-gux','\LTLGUX')
    fr = fr.replace('ltl3dra','\LTLGUX')
    fr = fr.replace('fg','\FG').replace('(','[$').replace(')','$]')
    if vertical:
        fr = '\rotatebox[origin=c]{90}{' + fr + '}'
    return fr

In [ ]:
def shorten_index(df, columns=True, cross=False,
                  inplace=False, vertical=False,
                  sb=False):
    if not inplace:
        df = df.copy()
    if cross:
        i = [(fix_tools(t[0]),t[1],fix_acc(t[2],sb),t[3]) for t in df.index.values]
    else:
        i = [(fix_tools(t[0]),t[1],fix_acc(t[2],sb)) for t in df.index.values]
    df.index=pd.MultiIndex.from_tuples(i)
    if columns:
        if cross:
            ci = [(fix_fragment(t[0],vertical),t[1]) for t in df.columns.values]
        else:
            ci = [(t[0],fix_fragment(t[1]),t[2]) for t in df.columns.values]
        df.columns=pd.MultiIndex.from_tuples(ci)
    if not inplace:
        return df

In [ ]:
def add_type_lines(filename,Safra=True,vertical=False,end=None):
    cline = '\cmidrule[\lightrulewidth]{2-'+ str(end) +'}' \
        if vertical else '\midrule'
    todo = [(r'(\\multirow\{2\}\{\*\}\{ltl2dpa\})',''),
            (r'(\\multirow\{3\}\{\*\}\{ltl2dstar\})','')]
    if not Safra:
        todo = [todo[0]]
    if vertical:
        todo = [(r"&\s+"+t[0], ' & ') for t in todo]
    for rplc, pref in todo:
        with open(filename) as f:
            lines = f.read()
            lines = re.sub(rplc,cline + pref + r'\1',lines)
        with open(filename,"w") as f1:
            f1.write(lines)

In [ ]:
def fix_lines(filename, end=12, vertical=False):
    end = str(end)
    todo = [(r"(\\multirow\{2\}\{\*\}\{ltl2dpa\})",1,''),
        (r"(& \\multirow\{2\}\{\*\}\{Spot\})",2,'')]
    if vertical:
        todo = [(r"&\s+"+t[0],t[1]+1, ' & ') for t in todo]
    for rplc, start, pref in todo:
        with open(filename) as f:
            lines = f.read()
        lines = re.sub(rplc,
                       "\cmidrule{" +
                       "{}-{}".format(start,end) +
                       "}" + pref + r"\1",
                       lines)
        with open(filename,"w") as f1:
            f1.write(lines)
    with open(filename) as f:
        lines = f.read()
    lines = lines.replace('cline','cmidrule')
    lines = lines.replace('$nan$','---')
    if vertical:
        rplc = "\cmidrule{1-" + str(end) + "}"
        lines = lines.replace(rplc,"\midrule[\heavyrulewidth]")
    # Remove cmidrules in front of midrule
    lines = re.sub(r"(\\cmidrule\{\d+-\d+\}\n?)*\\midrule\[\\heavyrulewidth\](\s*\n*\s*\\cmidrule\{\d+-\d+\}\s*\n?)*", "\\midrule[\\heavyrulewidth] ", lines)
    lines = re.sub(r"(\\cmidrule\{\d+-\d+\}\s*\n?)*(\\cmidrule\[\\lightrulewidth\]\{\d+-\d+\})(\s*\n*\s*\\cmidrule\{\d+-\d+\}\s*\n?)*", r"\2 ", lines)
    with open(filename,"w") as f1:
        f1.write(lines)

In [ ]:
def color_table(filename):
    setup = '''\\newcolumntype{a}{>{\\columncolor{blue!20}}r}
\\newcolumntype{b}{>{\\columncolor{blue!20}}c}
\\setlength{\\aboverulesep}{0pt}
\\setlength{\\belowrulesep}{0pt}
\\setlength{\\extrarowheight}{.75ex}
\\setlength{\\heavyrulewidth}{2pt}
\\setlength{\\lightrulewidth}{1.2pt}
\\def\\high{\\cellcolor{darkgreen!40}}
'''
    with open(filename) as f:
        lines = f.read()
    lines = fix_header_colors(lines)
    with open(filename,"w") as f1:
        f1.write(setup + lines)

In [ ]:
def make_heading(filename):
    with open(filename) as f:
        lines = f.read()
    lines = re.sub(r"(\s+)&\s+&\s+&\s+states", 
        r"\1main tool & intermediate & acc & states",
        lines)
    lines = re.sub(r"(\s+)&\s+&\s+&\s+&\s+0", 
        r"\1main tool & intermediate & acc & \# & 0",
        lines)
    with open(filename,"w") as f1:
        f1.write(lines)

In [ ]:
def fix_header_colors(lines):
    lines = lines.replace('{c}','{b}',1)
    i = lines.find('{c}')
    return lines[:i+1] + lines[i+1:].replace('{c}','{b}',1)

In [ ]:
def cummulative_to_latex(res,file,transpose=False,color=True):
    if transpose:
        res = res.T
    col_f = 'ccr'
    color_type = 'a' if color else 'r'
    for i in range(len(res.columns)//3):
        if i % 2 == 0:
            col_f += color_type*3
        else:
            col_f += 'rrr'
    res = res
    res.to_latex(buf=open(file,'w'), multirow=True,
         escape=False, na_rep='---',
         float_format=lambda x: '$' + '%0.0f' % x + '$',
         column_format=col_f, multicolumn_format='c')
    if color:
        color_table(file)

In [ ]:
def high_min(data):
    return ['\high ${:0.0f}$'.format(m) if 
            m == data.min() else m for m in data]

def high_max(data):
    return ['\high ${:0.0f}$'.format(m) if 
            m == data.max() else m for m in data]

In [ ]:
res = sort_by_tools(res,tool_order)
res = split_cols(res,axis=0,symbol='/')
shorten_index(res,inplace=True)
sorted_all = res.astype(float).apply(high_min)

In [ ]:
cumulative = [('random','rand'),('literature','lit')]
for long, short in cumulative:
    filename = 'cum_{}.tex'.format(short)
    cummulative_to_latex(sorted_all[long], file=filename)
    # We need to measure the # of columns in the LaTeX table
    length = len(sorted_all[long].columns) +\
             len(sorted_all[long].index.levels)
    fix_lines(filename,length)
    add_type_lines(filename)
    make_heading(filename)

## Cross-comparison

A tool `t1` wins against `t2` in the comparison also if `t2` fails and `t1` does not.

In [ ]:
def prepare_cross(df,victories=True):
    df = split_cols(df,axis=0,symbol='/')  
    df = df.T.reset_index(drop=True).T
    df = df.reset_index()
    df = df.set_index(['level_0','level_1','level_2'],
                      append=True)
    df = df.reorder_levels([1,2,3,0])
    df.index.names = ['','','','']
    if victories:
        cols = list(df.columns)
        cols[len(cols)-1] = 'V'
        df.columns = cols
    return df

In [ ]:
def cross_to_latex(res, file='cross.tex', color=True,
                   vertical=False):
    col_f = 'cccrr' if vertical else 'ccrr'
    color_type = 'a' if color else 'r'
    for i in range(len(res.columns)):
        if i % 2 == 0:
            col_f += color_type
        else:
            col_f += 'r'
    res = res
    res.to_latex(buf=open(file,'w'), multirow=True,
        escape=False,na_rep='---', 
        float_format=lambda x: '$' + '%0.0f' % x + '$',
        column_format=col_f,multicolumn_format='c')

In [ ]:
def create_cross(runners_base, tools, 
                 fragments=['full','ltl3dra'],
                 cols=['states','acc'],
                 vertical=False, merge_in_latex=True,
                 filename='cross.tex', latex=True):   
    def to_latex(table, filename=filename, vertical=False):
        table = table.copy()
        if vertical:
            data = []
            for k in table.index.levels[0]:
                data.append(
                    pd.DataFrame(table.loc[k,'V']).\
                    apply(high_max))
                vict = pd.concat(data,
                    keys=table.index.levels[0])
            table.loc[:,'V'] = vict
        else:
            table.loc(axis=1)[:,'V'] = \
                table.loc(axis=1)[:,'V'].apply(high_max)
        cross_to_latex(table, file=filename, 
                       vertical=vertical)
        end = len(table.columns) + len(table.index.levels)
        add_type_lines(filename, Safra=False, 
                       vertical=vertical, end=end)
        fix_lines(filename, end, vertical)
        color_table(filename)
        make_heading(filename)
    
    state_based = 'sb_states' in cols
    data = []
    for fr in fragments:
        r = runners['{}_{}'.format(runners_base,fr)]
        res = r.cross_compare(tools=tools,props=cols)
        cross = prepare_cross(res)
        data.append(cross)
        if latex and not merge_in_latex:
            table = pd.concat([cross], axis=1, keys=[fr])
            table = shorten_index(table,cross=True,
                        columns=False,inplace=False,
                        sb=state_based)
            fr_name = '{}_{}.tex'.format(filename[:-4],fr)
            to_latex(table, fr_name)
    table = pd.concat(data, axis=1, keys=fragments)
    shorten_index(table, cross=True, inplace=True,
                  vertical=vertical,
                  sb=state_based)
    if vertical:
        fragments = [fix_fragment(f, vertical=vertical)
                     for f in fragments]
        data = [table[f] for f in fragments]
        table = pd.concat(data,keys=fragments)
    if latex and merge_in_latex:
        to_latex(table,filename,vertical)
    return table

In [ ]:
direct = [t for t in tool_order if
          'LTL3DRA' in t or
          'R3' in t or 'R4' in t]
parity = [t for t in tool_order if 
          'ltl2dpa' in t or 'ltl2tgba' in t]
ltl2dstar = [t for t in tool_order if 'ltl2dstar' in t]
spot = [t for t in tool_order if 
        t.startswith('Spot') or
        'ltl2tgba' in t]
Safra_sel = [t for t in tool_order if 
           ('Spot/' in t and '/TGBA' in t) or
           'ltl2dstar/' in t or 
           'ltl2tgba' in t]
Rabin = [t for t in tool_order if 
           'ltl2tgba' in t or
           ('/DSRA' in t and 'R3' not in t) or
           'ltl2dpa' in t]
best = [t for t in tool_order if 
           'ltl2tgba' in t or
           'R4//DTGRA' in t or
           'ltl2dpa/ldba' in t or
           'LTL3TELA' in t]
toolsets = {
    'direct' : direct,
    'ltl2dstar' : ltl2dstar,
    'spot' : spot,
    'parity' : parity,
    'best' : best
}
toolsets_sbacc = {
    'Safra_sel' : Safra_sel,
    'Rabin' : Rabin,
}

In [ ]:
for name, tools in toolsets.items():
    filename = 'cross_lit_{}.tex'.format(name)
    create_cross('literature', tools, 
                 filename=filename, merge_in_latex=True,
                 vertical=True)
for name, tools in toolsets_sbacc.items():
    filename = 'cross_lit_{}.tex'.format(name)
    create_cross('literature', tools, cols=['sb_states','acc'],
                 filename=filename, merge_in_latex=True,
                 vertical=True)

In [ ]:
for name, tools in toolsets.items():
    filename = 'cross_rand_{}.tex'.format(name)
    create_cross('random', tools, 
                 filename=filename, merge_in_latex=True,
                 vertical=True,fragments=['full','ltl3dra','fg'])
for name, tools in toolsets_sbacc.items():
    filename = 'cross_rand_{}.tex'.format(name)
    create_cross('random', tools, cols=['sb_states','acc'],
                 filename=filename, merge_in_latex=True,
                 vertical=True,fragments=['full','ltl3dra','fg'])

In [ ]:
r = runners['literature_ltl3dra']
s = r.values.states.dropna()
t1 = 'ltl2tgba//DPA'
t2 = 'R4//DSRA'
t3 = 'ltl2dpa/ldba/DTPA'
val1 = sorted(list(s.loc(axis=1)[t1]))
val2 = sorted(list(s.loc(axis=1)[t2]))
val3 = sorted(list(s.loc(axis=1)[t3]))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import seaborn

fig = plt.figure(figsize=(6, 6))

plt.yticks(fontsize=12)
plt.xticks(fontsize=12)

msize = 12
LW=3
trace1, = plt.plot(range(0, len(val1)), val1, '-', linewidth=LW, ms = msize, color='green')
trace2, = plt.plot(range(0, len(val2)), val2, ':', linewidth=LW, markerfacecolor='none', ms = msize, color='blue')
trace3, = plt.plot(range(0, len(val3)), val3, '--', linewidth=LW, ms=msize, color='red')
plt.yscale('log')
#trace4, = plt.plot(range(0, len(val4)), val4, '-^', markerfacecolor='none', ms=msize)
#plt.yscale('log')
#plt.setp(color='red')
plt.xlabel('n-th fastest benchmark', fontsize=16, labelpad=8)
plt.ylabel('time [s]', fontsize=14, labelpad=10)
plt.legend([trace1, trace2, trace3],
           [t1,t2,t3],
            fontsize=14)

pp = PdfPages('plot.pdf')
pp.savefig(fig, bbox_inches='tight')
pp.close()
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 10))

plt.yticks(fontsize=12)
plt.xticks(fontsize=12)

msize = 12
LW=3

plt.plot(s.index.labels[0],s[t1], '*')
plt.plot(s.index.labels[0],s[t2], '*')
plt.plot(s.index.labels[0],s[t3], '*')
plt.yscale('log')

In [ ]:
s.plot.line()

In [ ]:
val1

In [ ]:
r.cummulative('states')

In [ ]:
r.cummulative('sb_states')

In [ ]:
r.cross_compare(Safra,['sb_states','acc'])